In [96]:
import nltk
import re
from nltk.corpus import wordnet as wn
from wordfreq import zipf_frequency
from nltk.tokenize import MWETokenizer
from nltk.tokenize import word_tokenize
import spacy

nlp = spacy.load("en_core_web_sm")

import inflect

p = inflect.engine()

nb = [x.name() for x in wn.all_synsets()]
mwe = []
for x in nb:
    if '_' in x:
        mwe.append(tuple(x.split('.')[0].split('_'))) 
mwe.append(tuple(x.split('.')[0].split('_')))
tokenizer = MWETokenizer(mwe)

In [97]:
nouns = ['NN','NNS']

In [106]:
def lexical_simp(s):
    s = s.replace('.',' .')# add a space before every punctuation ". , ; ! : ? etc."
    s = s.replace('!',' !')
    s = s.replace(':', ' :')
    s = s.replace(';', ' ;')
    s = s.replace(',',' ,')
    t = tokenizer.tokenize(s.split())
    Ntag = ['NN', 'NNS']
    adjtag = ['JJ', 'JJR','JJS']
    #search for comma and adj before multiword and take simple word
    index = 0
    sentence = []
    for i in t:
        if i == ',':
            token = nlp(t[index-1])
            sentence.append(i)
            if token[0].tag_ in adjtag:
                token = t[index+1].replace('_',' ')
                token = word_tokenize(token)
                for j in token:
                    sentence.append(j)
                t.pop(index)
        else:
            sentence.append(i)
        index+=1
    # simplification'
    t = sentence
    for word in t: # 'I wear a titfer'
        if nltk.pos_tag([word])[0][1] in Ntag: # titfers (NNS) True
            if zipf_frequency(word, 'en') < 2.5: #should be simplified
                plural = False
                if nltk.pos_tag([word])[0][1] == 'NNS' or p.singular_noun(word) != False:
                    plural = True
                    word = p.singular_noun(word)
                w = wn.synsets(word)
                if w != []:
                    hypernym = []

                    for i in w:
                        hypernym.extend(i.hypernym_paths()[0])

                    n = re.match(r'^(.+)\.[a-z]+\.[0-9]+', w[0].name())
                    word_f = [((zipf_frequency(n.group(1), 'en'), n.group(1)))]
                    proposition = []

                    for j in hypernym[-5:]:
                        name = j.name().replace('_',' ')
                        m = re.match(r'^(.+)\.[a-z]+\.[0-9]+', name)
                        proposition.append(m.group(1))

                    candidate = []
                    for k in proposition:
                        candidate.append((zipf_frequency(k, 'en'),k))

                    choice = sorted(candidate + word_f, reverse = True)
                    b = choice[0][1]
                    if plural == True:
                        b = p.plural_noun(choice[0][1])
                        word = p.plural_noun(word)
                    t = [a.replace(word,b) for a in t]

    index = 0
    vowel = ['a','e','i','o','u']
    for i in t:
        if i == 'a':
            if t[index+1][0] in vowel:
                t[index] = 'an'
        elif i == 'an':
            if t[index+1][0] not in vowel:
                t[index] = 'a'
        index+=1 
    
    
    s = ' '.join(t)
    s = s.replace("_", ' ')
    s = s.replace(' .','.')
    s = s.replace(' !','!')
    s = s.replace(' :', ':')
    s = s.replace(' ;', ';')
    s = s.replace(' ,', ',')
    
    return s # 'I wear a hat'

In [107]:
s1 = 'I have two corgis and a titfer.'
s2 = 'You are visiting many beautiful castles.'
s3 = 'I love salmon.'
s4 = "We see a big and tall man. He likes wearing tons of titfers."
s5 = 'This is a real golden retriever !'
s6 = 'He did an abnegation of what I said.'
s7 = 'The rebel officers who led the abortive coup were shot'

s8 = "I have two corgis and a bike. You are visiting many beautiful castles. I love salmon. We see a big and tall man. He likes wearing tons of titfers. This is a real golden retriever ! He did an abnegation of what I said. The rebel officers who led the abortive coup were shot."
#s8 = "I have two corgis and a bike. He likes wearing tons of titfers. This is a real golden retriever !"

#lexical_simp(s1)
#lexical_simp(s2)
#lexical_simp(s3)
#lexical_simp(s4)
#lexical_simp(s5)
#lexical_simp(s6)
#lexical_simp(s7)
lexical_simp(s8)

'I have two dogs and a bike. You are visiting many beautiful castles. I love salmon. We see a big and tall man. He likes wearing tons of hats. This is a real dog! He did an act of what I said. The rebel officers who led the abortive coup were shot.'

In [89]:
test = "Early Wisdom Literature from the Fertile Crescent was a genre that sought to instruct people on ethical action, practical living, and virtue through stories and proverbs. In Ancient Egypt, these texts were known as  ('teachings') and they are central to our understandings of Ancient Egyptian philosophy."
lexical_simp(test) #no change

INITIAL SENTENCE:
 Early Wisdom Literature from the Fertile Crescent was a genre that sought to instruct people on ethical action, practical living, and virtue through stories and proverbs. In Ancient Egypt, these texts were known as  ('teachings') and they are central to our understandings of Ancient Egyptian philosophy.


"Early Wisdom Literature from the Fertile Crescent was a genre that sought to instruct people on ethical action, practical living, and virtue through stories and proverbs. In Ancient Egypt, these texts were known as ('teachings') and they are central to our understandings of Ancient Egyptian philosophy."

In [90]:
# the sentence don't mean anything it's just to test everything
s = 'the hat is an wide, flat bone that is not good titfer and golden retriever, it even have a astrological sign'

In [91]:
lexical_simp(s)

INITIAL SENTENCE:
 the hat is an wide, flat bone that is not good titfer and golden retriever, it even have a astrological sign


'the hat is a wide, flat bone that is not good hat and dog, it even have an astrological sign'